In [1]:
!apt-get --purge remove cuda nvidia* libnvidia-*
!dpkg -l | grep cuda- | awk '{print $2}' | xargs -n1 dpkg --purge
!apt-get remove cuda-*
!apt autoremove
!apt-get update

Reading package lists... Done
Building dependency tree       
Reading state information... Done
Note, selecting 'nvidia-kernel-common-418-server' for glob 'nvidia*'
Note, selecting 'nvidia-325-updates' for glob 'nvidia*'
Note, selecting 'nvidia-346-updates' for glob 'nvidia*'
Note, selecting 'nvidia-driver-binary' for glob 'nvidia*'
Note, selecting 'nvidia-331-dev' for glob 'nvidia*'
Note, selecting 'nvidia-304-updates-dev' for glob 'nvidia*'
Note, selecting 'nvidia-compute-utils-418-server' for glob 'nvidia*'
Note, selecting 'nvidia-384-dev' for glob 'nvidia*'
Note, selecting 'nvidia-libopencl1-346-updates' for glob 'nvidia*'
Note, selecting 'nvidia-driver-440-server' for glob 'nvidia*'
Note, selecting 'nvidia-340-updates-uvm' for glob 'nvidia*'
Note, selecting 'nvidia-dkms-450-server' for glob 'nvidia*'
Note, selecting 'nvidia-kernel-common' for glob 'nvidia*'
Note, selecting 'nvidia-kernel-source-440-server' for glob 'nvidia*'
Note, selecting 'nvidia-331-updates-uvm' for glob 'nvidi

In [2]:
!wget https://developer.nvidia.com/compute/cuda/9.2/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64 -O cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!dpkg -i cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!apt-key add /var/cuda-repo-9-2-local/7fa2af80.pub
!apt-get update
!apt-get install cuda-9.2

--2021-03-07 09:58:17--  https://developer.nvidia.com/compute/cuda/9.2/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64
Resolving developer.nvidia.com (developer.nvidia.com)... 152.199.0.24
Connecting to developer.nvidia.com (developer.nvidia.com)|152.199.0.24|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://developer.nvidia.com/compute/cuda/9.2/prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64 [following]
--2021-03-07 09:58:17--  https://developer.nvidia.com/compute/cuda/9.2/prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64
Reusing existing connection to developer.nvidia.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://developer.download.nvidia.com/compute/cuda/9.2/secure/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb?4NfSIGYNL4GmqHnG5ALGcYyNeF5wwMFpPUJ5YC4a6oZ1qn2zFMMdPJqxZn7RVufFV-NdjUvxD8lPl7W8FL5eh1cuZWxCe7knkm_C2--2cUUa-l0Zd

In [3]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2018 NVIDIA Corporation
Built on Wed_Apr_11_23:16:29_CDT_2018
Cuda compilation tools, release 9.2, V9.2.88


In [4]:
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git

  Cloning git://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-j_oefny6
  Running command git clone -q git://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-j_oefny6
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-cp37-none-any.whl size=4307 sha256=4fc501745cf0ac228c8e19b6b9e5696777cea6e2488cc2fba58faf1f72f7019e
  Stored in directory: /tmp/pip-ephem-wheel-cache-9lw_fgia/wheels/10/c2/05/ca241da37bff77d60d31a9174f988109c61ba989e4d4650516
Successfully built NVCCPlugin


In [5]:
%load_ext nvcc_plugin

created output directory at /content/src
Out bin /content/result.out


In [32]:
%%cu
#include <curand_kernel.h>
#include <stdio.h>
#include <time.h>
#include <string.h>

#define LLI_MAX 0x7fffffffffffffff

#include <stdlib.h>

#define K 3
#define N 16

__global__ void create_i_centers_mat(unsigned long seed, bool *i_centers_mat, float proba, int start, int k)
{
    int tid = threadIdx.x;

    curandState state;
    curand_init(seed, tid, 0, &state);
 
    i_centers_mat[(tid + start) * (k+1) + 0] = 1;
    bool dropped = false;
    for (int a = 1; a < k; a++)
    {
        float ranv = curand_uniform(&state);
        i_centers_mat[(tid + start) * (k+1) + a] = !dropped && (ranv <= proba);
        if (!i_centers_mat[(tid + start) * (k+1) + a])
        {
            dropped = true;
        }
    }
    i_centers_mat[(tid + start) * (k+1) + k] = 0;
}

__global__ void create_i_centers_min_dist_vert__bunches(int *i_centers_min_dist_vert, int start, int k, int n, long long int *dist_mat, int *i_centers, int *i_centers_len)
{
    int tid = threadIdx.x;
    int vertex = tid + start;
  
    for (int a = k; a > 0; a--)
    {
        long long int min_distance = LLI_MAX;
        for (int b = 0; b < i_centers_len[a]; b++)
        {
            if (dist_mat[n*vertex + i_centers[n*a + b]] < min_distance)
            {
                min_distance = dist_mat[n*vertex + i_centers[n*a + b]];
                i_centers_min_dist_vert[n*a + vertex] = i_centers[n*a + b];
            }
        }
    }
    i_centers_min_dist_vert[n*0 + vertex] = vertex;
}

int main()
{
    bool i_centers_mat[N][K+1], *i_centers_mat_dev;
    cudaMalloc(&i_centers_mat_dev, N*(K+1)*sizeof(bool));
 
    // CALCULATE PROBA
    float proba = 0.5;
    create_i_centers_mat<<<1,N>>>(time(0), i_centers_mat_dev, proba, 0, K);
    cudaDeviceSynchronize();
 
    cudaMemcpy(&i_centers_mat, i_centers_mat_dev, N*(K+1)*sizeof(bool), cudaMemcpyDeviceToHost);
    cudaFree(i_centers_mat_dev);
 
    for (int a = 0; a < N; a++)
    {
        printf("Vertex %d\t: ", a);
        for (int b = 0; b <= K; b++)
        {
            printf("%d ", i_centers_mat[a][b]);
        }
        printf("\n");
    }
    printf("\n");
  
    int i_centers[K+1][N], i_centers_len[K+1];
    memset(&i_centers, 0, sizeof(i_centers));
    memset(&i_centers_len, 0, sizeof(i_centers_len));
 
    int i_centers_diff[K][N], i_centers_diff_len[K];
    memset(&i_centers_diff, 0, sizeof(i_centers_diff));
    memset(&i_centers_diff_len, 0, sizeof(i_centers_diff_len));

    for(int a = 0; a < N; a++)
    {
        for (int b = 0; b <= K; b++)
        {
            if (i_centers_mat[a][b])
            {
                i_centers[b][i_centers_len[b]++] = a;
            }
            else
            {
                i_centers_diff[b-1][i_centers_diff_len[b-1]++] = a;
                break;
            }
        }
    }

    int *i_centers_dev, *i_centers_len_dev;
    cudaMalloc(&i_centers_dev, (K+1)*N*sizeof(int));
    cudaMemcpy(i_centers_dev, &i_centers, (K+1)*N*sizeof(int), cudaMemcpyHostToDevice);
    cudaMalloc(&i_centers_len_dev, (K+1)*sizeof(int));
    cudaMemcpy(i_centers_len_dev, &i_centers_len, (K+1)*sizeof(int), cudaMemcpyHostToDevice);
 
    int *i_centers_diff_dev, *i_centers_diff_len_dev;
    cudaMalloc(&i_centers_diff_dev, K*N*sizeof(int));
    cudaMemcpy(i_centers_diff_dev, &i_centers_diff, K*N*sizeof(int), cudaMemcpyHostToDevice);
    cudaMalloc(&i_centers_diff_len_dev, K*sizeof(int));
    cudaMemcpy(i_centers_diff_len_dev, &i_centers_diff_len, K*sizeof(int), cudaMemcpyHostToDevice);

    for (int a = 0; a <= K; a++)
    {
        printf("I Center %d\t: ", a);
        for (int b = 0; b < i_centers_len[a]; b++)
        {
            printf("%d ", i_centers[a][b]);
        }
        printf("\n");
    }
    printf("\n");
    for (int a = 0; a < K; a++)
    {
        printf("I Center Diff %d - %d\t: ", a, a+1);
        for (int b = 0; b < i_centers_diff_len[a]; b++)
        {
            printf("%d ", i_centers_diff[a][b]);
        }
        printf("\n");
    }
    printf("\n");
 
    // SOLVE APSP
    srand(time(0));
    long long int dist_mat[N][N], *dist_mat_dev;
    for (int a = 0; a < N; a++)
    {
        dist_mat[a][a] = 0;
        for (int b = 0; b < a; b++)
        {
            dist_mat[a][b] = rand()%46 + 5;
            dist_mat[b][a] = dist_mat[a][b];
        }
    }
    for (int a = 0; a < N; a++)
    {
        printf("\tV%d", a);
    }
    printf("\n");
    for (int a = 0; a < N; a++)
    {
        printf("V%d\t", a);
        for (int b = 0; b < N; b++)
        {
            printf("%lld\t", dist_mat[a][b]);
        }
        printf("\n");
    }
    printf("\n");
    cudaMalloc(&dist_mat_dev, N*N*sizeof(long long int));
    cudaMemcpy(dist_mat_dev, &dist_mat, N*N*sizeof(long long int), cudaMemcpyHostToDevice);
 
    int i_centers_min_dist_vert[K][N], *i_centers_min_dist_vert_dev;
    cudaMalloc(&i_centers_min_dist_vert_dev, K*N*sizeof(int));
   
    int bunches[N][N], bunches_len[N], *bunches_dev, *bunches_len_dev;
    cudaMalloc(&bunches_dev, N*N*sizeof(int));
    cudaMalloc(&bunches_len_dev, N*sizeof(int));

    // Create bunches
    create_i_centers_min_dist_vert__bunches<<<1,N>>>(i_centers_min_dist_vert_dev, 0, K, N, dist_mat_dev, i_centers_dev, i_centers_len_dev);
    cudaDeviceSynchronize();
 
    cudaFree(dist_mat_dev); // MAY NEED TO BE CHANGED BASED ON APSP CHANGES
    cudaFree(i_centers_dev);
    cudaFree(i_centers_len_dev);
    cudaFree(i_centers_diff_dev);
    cudaFree(i_centers_diff_len_dev);
 
    cudaMemcpy(&i_centers_min_dist_vert, i_centers_min_dist_vert_dev, K*N*sizeof(int), cudaMemcpyDeviceToHost);
    cudaFree(i_centers_min_dist_vert_dev);
    
    cudaMemcpy(&bunches, bunches_dev, N*N*sizeof(int), cudaMemcpyDeviceToHost);
    cudaFree(bunches_dev);
    cudaMemcpy(&bunches_len, bunches_len_dev, N*sizeof(int), cudaMemcpyDeviceToHost);
    cudaFree(bunches_len_dev);
 

    for (int a = 0; a < N; a++)
    {
        printf("Vertex %d\t: ", a);
        for (int b = 0; b < K; b++)
        {
            printf("%d ", i_centers_min_dist_vert[b][a]);
        }
        printf("\n");
    }
    printf("\n");
 
    // Create hash from bunches
}

Vertex 0	: 1 1 0 0 
Vertex 1	: 1 0 0 0 
Vertex 2	: 1 1 1 0 
Vertex 3	: 1 0 0 0 
Vertex 4	: 1 0 0 0 
Vertex 5	: 1 1 1 0 
Vertex 6	: 1 0 0 0 
Vertex 7	: 1 0 0 0 
Vertex 8	: 1 1 0 0 
Vertex 9	: 1 0 0 0 
Vertex 10	: 1 1 0 0 
Vertex 11	: 1 1 1 0 
Vertex 12	: 1 0 0 0 
Vertex 13	: 1 1 1 0 
Vertex 14	: 1 0 0 0 
Vertex 15	: 1 1 1 0 

I Center 0	: 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 
I Center 1	: 0 2 5 8 10 11 13 15 
I Center 2	: 2 5 11 13 15 
I Center 3	: 

I Center Diff 0 - 1	: 1 3 4 6 7 9 12 14 
I Center Diff 1 - 2	: 0 8 10 
I Center Diff 2 - 3	: 2 5 11 13 15 

	V0	V1	V2	V3	V4	V5	V6	V7	V8	V9	V10	V11	V12	V13	V14	V15
V0	0	46	20	41	22	14	19	41	8	13	30	29	46	18	21	46	
V1	46	0	42	11	33	25	38	30	15	44	49	9	8	32	5	48	
V2	20	42	0	15	9	42	48	35	18	12	47	44	25	13	35	9	
V3	41	11	15	0	10	49	20	37	49	17	21	41	8	12	38	46	
V4	22	33	9	10	0	49	14	36	30	49	12	12	21	22	46	15	
V5	14	25	42	49	49	0	30	24	49	21	5	48	24	5	16	36	
V6	19	38	48	20	14	30	0	17	33	32	41	48	18	37	27	48	
V7	41	30	35	37	36	24	17	0	30	40	49	